In [1]:
import pandas as pd

df = pd.read_csv("train.csv")

In [2]:
df.head()

,id,latitude,longitude,day_of_year,day_of_week,hour,month,pollution_value
0,0,51.491,-0.172,301,6,3,10,14.5
1,1,39.386,-121.158,254,3,21,9,34.5
2,2,51.459,0.596,301,6,3,10,10.5
3,3,35.299,-120.613,145,2,14,5,15.5
4,4,29.927,120.527,221,0,14,8,54.5


In [10]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 51.491,
	"longitude": -0.172,
	"start_date": "2024-10-12",
	"end_date": "2024-11-11",
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", "wind_direction_100m", "precipitation", "relative_humidity_2m", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high"],
	"timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_direction_100m = hourly.Variables(4).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(5).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(6).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(7).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(8).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(9).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(11).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 51.49384689331055°N -0.16302490234375°E
Elevation 12.0 m asl
Timezone b'Europe/London'b'GMT+1'
Timezone difference to GMT+0 3600 s
                         date  temperature_2m  wind_speed_10m  \
0   2024-10-11 23:00:00+00:00           8.328       13.633737   
1   2024-10-12 00:00:00+00:00           8.028       12.601285   
2   2024-10-12 01:00:00+00:00           8.778       13.551501   
3   2024-10-12 02:00:00+00:00           8.828       14.142320   
4   2024-10-12 03:00:00+00:00           9.128       15.913465   
..                        ...             ...             ...   
739 2024-11-11 18:00:00+00:00           7.278       12.534257   
740 2024-11-11 19:00:00+00:00           6.828       12.754622   
741 2024-11-11 20:00:00+00:00           6.328       11.989945   
742 2024-11-11 21:00:00+00:00           5.978       12.221456   
743 2024-11-11 22:00:00+00:00           5.828       12.576836   

     wind_speed_100m  wind_direction_10m  wind_direction_100m  precipitation

In [11]:
hourly_dataframe

,date,temperature_2m,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,precipitation,relative_humidity_2m,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high
0,2024-10-11 23:00:00+00:00,8.328,13.633737,29.022451,137.675491,144.338318,0.0,67.581245,1013.123535,100.0,2.0,4.0,100.0
1,2024-10-12 00:00:00+00:00,8.028,12.601285,28.449112,135.578644,141.679840,0.0,70.204437,1012.922302,100.0,10.0,8.0,100.0
2,2024-10-12 01:00:00+00:00,8.778,13.551501,28.465620,129.610657,139.359497,0.0,68.882332,1012.127502,100.0,2.0,37.0,100.0
3,2024-10-12 02:00:00+00:00,8.828,14.142320,29.007380,137.579193,145.619598,0.0,70.611633,1011.528381,100.0,9.0,64.0,100.0
4,2024-10-12 03:00:00+00:00,9.128,15.913465,31.303444,142.352310,149.601349,0.0,71.919640,1010.531433,100.0,9.0,92.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2024-11-11 18:00:00+00:00,7.278,12.534257,27.645092,347.560455,351.764465,0.0,84.144691,1035.385620,0.0,0.0,0.0,0.0
740,2024-11-11 19:00:00+00:00,6.828,12.754622,28.177332,351.069489,355.236450,0.0,84.386879,1035.882202,0.0,0.0,0.0,0.0
741,2024-11-11 20:00:00+00:00,6.328,11.989945,27.407917,352.234924,356.611511,0.0,85.821884,1035.879639,0.0,0.0,0.0,0.0
742,2024-11-11 21:00:00+00:00,5.978,12.221456,27.598761,350.676483,356.260559,0.0,87.611404,1035.777832,0.0,0.0,0.0,0.0


In [9]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
from datetime import datetime, timedelta
from tqdm import tqdm
import os

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Weather outputs klasörü oluşturulmamışsa oluştur
os.makedirs("weather_outputs", exist_ok=True)

# ID'ye göre döngü
for _, row in tqdm(df.iterrows(), total=len(df)):
    id_ = row["id"]
    lat = row["latitude"]
    lon = row["longitude"]
    day_of_year = int(row["day_of_year"])

    output_path = f"weather_outputs/id_{id_}.csv"
    if os.path.exists(output_path):
        print(f"[ID: {id_}] Zaten mevcut, atlanıyor.")
        continue

    # Tarih hesapla
    base_date = datetime(year=2024, month=1, day=1)
    original_date = base_date + timedelta(days=day_of_year - 1)
    start_date = (original_date - timedelta(days=15)).strftime("%Y-%m-%d")
    end_date = (original_date + timedelta(days=15)).strftime("%Y-%m-%d")

    # API parametreleri
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m", "wind_speed_10m", "wind_speed_100m",
            "wind_direction_10m", "wind_direction_100m", "precipitation",
            "relative_humidity_2m", "surface_pressure", "cloud_cover",
            "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high"
        ],
        "timezone": "auto"
    }

    try:
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)
        response = responses[0]
        hourly = response.Hourly()

        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(1).ValuesAsNumpy(),
            "wind_speed_100m": hourly.Variables(2).ValuesAsNumpy(),
            "wind_direction_10m": hourly.Variables(3).ValuesAsNumpy(),
            "wind_direction_100m": hourly.Variables(4).ValuesAsNumpy(),
            "precipitation": hourly.Variables(5).ValuesAsNumpy(),
            "relative_humidity_2m": hourly.Variables(6).ValuesAsNumpy(),
            "surface_pressure": hourly.Variables(7).ValuesAsNumpy(),
            "cloud_cover": hourly.Variables(8).ValuesAsNumpy(),
            "cloud_cover_low": hourly.Variables(9).ValuesAsNumpy(),
            "cloud_cover_mid": hourly.Variables(10).ValuesAsNumpy(),
            "cloud_cover_high": hourly.Variables(11).ValuesAsNumpy(),
        }

        hourly_df = pd.DataFrame(hourly_data)
        hourly_df.to_csv(output_path, index=False)

    except Exception as e:
        print(f"[ID: {id_}] Veri çekilirken hata oluştu: {e}")

  0%|          | 0/7649 [00:00<?, ?it/s]

[ID: 0.0] Zaten mevcut, atlanıyor.
[ID: 1.0] Zaten mevcut, atlanıyor.
[ID: 2.0] Zaten mevcut, atlanıyor.
[ID: 3.0] Zaten mevcut, atlanıyor.
[ID: 4.0] Zaten mevcut, atlanıyor.
[ID: 5.0] Zaten mevcut, atlanıyor.
[ID: 6.0] Zaten mevcut, atlanıyor.
[ID: 7.0] Zaten mevcut, atlanıyor.
[ID: 8.0] Zaten mevcut, atlanıyor.
[ID: 9.0] Zaten mevcut, atlanıyor.
[ID: 10.0] Zaten mevcut, atlanıyor.
[ID: 11.0] Zaten mevcut, atlanıyor.
[ID: 12.0] Zaten mevcut, atlanıyor.
[ID: 13.0] Zaten mevcut, atlanıyor.
[ID: 14.0] Zaten mevcut, atlanıyor.
[ID: 15.0] Zaten mevcut, atlanıyor.
[ID: 16.0] Zaten mevcut, atlanıyor.
[ID: 17.0] Zaten mevcut, atlanıyor.
[ID: 18.0] Zaten mevcut, atlanıyor.
[ID: 19.0] Zaten mevcut, atlanıyor.
[ID: 20.0] Zaten mevcut, atlanıyor.
[ID: 21.0] Zaten mevcut, atlanıyor.
[ID: 22.0] Zaten mevcut, atlanıyor.
[ID: 23.0] Zaten mevcut, atlanıyor.
[ID: 24.0] Zaten mevcut, atlanıyor.
[ID: 25.0] Zaten mevcut, atlanıyor.
[ID: 26.0] Zaten mevcut, atlanıyor.
[ID: 27.0] Zaten mevcut, atlanıyor.
[I

  6%|▌         | 466/7649 [00:00<00:04, 1516.83it/s]

[ID: 465.0] Veri çekilirken hata oluştu: {'reason': 'Hourly API request limit exceeded. Please try again in the next hour.', 'error': True}
[ID: 466.0] Veri çekilirken hata oluştu: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}
[ID: 467.0] Veri çekilirken hata oluştu: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}
[ID: 468.0] Veri çekilirken hata oluştu: {'reason': 'Hourly API request limit exceeded. Please try again in the next hour.', 'error': True}
[ID: 469.0] Veri çekilirken hata oluştu: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}
[ID: 470.0] Veri çekilirken hata oluştu: {'reason': 'Hourly API request limit exceeded. Please try again in the next hour.', 'error': True}
[ID: 471.0] Veri çekilirken hata oluştu: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}
[ID: 472.0] Veri çek

  8%|▊         | 606/7649 [00:11<02:45, 42.57it/s]  

...

 16%|█▌        | 1221/7649 [00:40<08:14, 13.01it/s]

[ID: 1219.0] Veri çekilirken hata oluştu: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}
[ID: 1220.0] Veri çekilirken hata oluştu: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}
[ID: 1221.0] Veri çekilirken hata oluştu: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}


 16%|█▌        | 1225/7649 [00:40<03:34, 29.98it/s]

[ID: 1222.0] Veri çekilirken hata oluştu: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}
[ID: 1224.0] Veri çekilirken hata oluştu: {'error': True, 'reason': 'Hourly API request limit exceeded. Please try again in the next hour.'}


KeyboardInterrupt: 

In [21]:
df["latitude"].isna().sum()

np.int64(13)

In [24]:
df["longitude"].isna().sum()

np.int64(13)

In [25]:
df = df.dropna(subset=['latitude', 'longitude'])

In [26]:
df["latitude"].isna().sum()

np.int64(0)

In [27]:
df["longitude"].isna().sum()

np.int64(0)

In [28]:
df.to_csv("airpol.csv")